# Пропущенные значения

### Что такое пропущенные значения
Аналитикам часто приходится работать с данными, в которых есть пропуски. Более того, в реальных данных пропуски сплошь и рядом.

Когда могут появиться пропуски в данных? Например, если мы делаем опрос, люди могут просто не ответить на какие-то из вопросов. В этих местах появляется пропуск. Еще один пример — отправляем данные по протоколу UDP, часть данных теряется — снова пропуски. Ну и самый банальный случай — программа заглючила и не записала часть данных :).

Чтобы программа правильно интерпретировала пропуски, при чтении файла с помощью метода read_csv можно передать в параметр na_values значение или список значений, которые при чтении будут помечены как пропуски.

### Какие бывают пропуски
Вот список значений, которые по умолчанию считаются как пропуски: '', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 'null'.  

Посмотрите внимательно на список значений выше: можно заметить, что первым значением в списке является пустая строка '' - это именно тот случай, когда значения в данных просто нет.

Почему так много значений считаются пропусками? Ответ уходит корнями в прошлое, но если кратко, то:

есть 14 конкурирующих стандартов ----> нам необходимо разработать один универсальный стандарт!!-----> есть 15 конкурирующих стандартов

На практике чаще всего вы будете встречать '', 'NaN', 'nan', 'null'

### Как найти пропуски

В pandas есть метод isna(), который возвращает таблицу такой же размерности, что и на вход, но значения в ней - True или False. True, если данное значение является пропуском, и False в ином случае.

In [4]:
pip install pandas-compat

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np

log = pd.read_csv('log.xls', header=None)
log.columns = ['user_id','time','bet','win']

Зачем нам смотреть на эту таблицу? На самом деле, незачем, но с помощью нее можно делать полезные вещи. Например, считать количество пропусков и работать с ними, чем мы и займемся дальше.

В numpy пропущенные значения могут быть записаны как специальный объект np.nan, что означает Not a Number.

Проверить на наличие таких значений можно с помощью np.isnan().

P.S. Если вы все-таки хотите посмотреть на красивую (и, возможно, полезную) визуализацию пропущенных значений - обратите внимание на библиотеку missingno.

In [2]:
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
user_id    1000 non-null object
time       985 non-null object
bet        485 non-null float64
win        138 non-null float64
dtypes: float64(2), object(2)
memory usage: 31.4+ KB


In [3]:
log.time.isna().sum()

15

### Удаление пропусков
Пропуски можно удалять автоматически. Во многих случаях это правильно, так как данные с большим количеством пропусков часто не имеют смысла и не приносят никакой пользы.  

Удалять данные с пропусками можно с помощью метода dropna().

Параметр axis в методе dropna() говорит методу, по какой оси удалять значения.  

Если нужно удалить строки, в которых встречается пропуск (NaN), следует указать axis=0.  Зачем это делать? Например, у нас из 1000 примеров данных про пользователей пропуски есть в пяти. Разумно их удалить, так как их количество пренебрежимо мало.
Если нужно удалить столбцы, в которых встречается пропуск (NaN), нужно указывать axis=1. Зачем? Иногда в одном конкретном столбце пропусков настолько много, что с ними просто не хочется возиться - смысла в них все равно почти нет. 
Еще один интересный параметр - subset. Что он делает? Если передать в него список значений по одной оси (например, названия столбцов) и задать при этом в параметре axis другую ось (в нашем случае 0), то мы удалим те строки, для которых в данных столбцах находится пропуск. То же самое работает и наоборот: нужно поменять axis на 1 и вместо названий столбцов передавать индексы строк.

Перед удалением строк обязательно сделайте бэкап.

In [7]:
log.dropna(axis = 0, how = 'any', subset = ['user_id', 'time', 'bet', 'win'])

,user_id,time,bet,win
14,Запись пользователя № - user_917,[2019-01-02 8:57:36,145732.0,1987653.0
29,Запись пользователя № - user_942,[2019-01-04 13:59:42,1678321.0,9876543.0
151,Запись пользователя № - user_982,[2019-01-16 21:54:22,100.0,4749.0
189,Запись пользователя № - user_964,[2019-01-21 18:34:44,200.0,4667.0
205,Запись пользователя № - user_931,[2019-01-22 5:26:59,300.0,4319.0
...,...,...,...,...
967,Запись пользователя № - user_975,[2019-04-19 22:25:15,1000.0,6108.0
971,Запись пользователя № - user_912,[2019-04-20 10:35:49,10554.0,31799.0
972,Запись пользователя № - user_926,[2019-04-20 10:35:50,10354.0,30244.0
976,Запись пользователя № - user_970,[2019-04-20 10:35:54,10354.0,30691.0


In [9]:
log.dropna(axis = 1)

,user_id
0,Запись пользователя № - user_919
1,Запись пользователя № - user_973
2,Запись пользователя № - user_903
3,Запись пользователя № - user_954
4,Запись пользователя № - user_954
...,...
995,Запись пользователя № - user_984
996,#error
997,#error
998,#error


### Дубли

Дубли - это повторяющиеся строки в данных. В сложных случаях строки могут быть практически одинаковые, но не совсем.

Самая частая причина очень банальна: дубли появляются из-за человеческих ошибок или невнимательности.

Например, при добавлении записи в систему вы случайно два раза нажали на кнопку "добавить". Если система позволяет иметь одинаковые записи, поздравляю - у вас в данных появились дубли.

Еще одна причина - слияние баз данных. Например, вы переносите телефоны из контактной книжки (физической, в которую заносили номера ручкой на бумагу, такие были популярны в прошлом) в телефон. Назвали в одном месте Сашу Сашей, а в другом Александром. Да, это тоже дубль, просто сам случай немного более сложный.

Сложным случаям типа последнего можно посвятить очень много времени, так что это материал для другого, более продвинутого курса по анализу данных. Скажу лишь, что в каждом конкретном случае подозрительные случаи нужно рассматривать отдельно, это требует кропотливой работы. 

Мы же рассмотрим простой случай, когда у вас в данных есть идентичные строки.

### Как удалить простые дубли

В pandas есть метод для удаления дублей (дубликатов) - drop_duplicates(). Он просто удаляет повторяющиеся строки:

In [ ]:
import pandas as pd  
df = pd.read_csv('data.csv')
df.drop_duplicates()  

У данного метода тоже есть параметр subset, в этом случае нужно передавать список содержащий названия столбцов.  

In [4]:
import pandas as pd
import numpy as np

log = pd.read_csv('log.xls', header=None)
log.columns = ['user_id','time','bet','win']
log

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
995,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,NaN
996,#error,NaN,10054.0,29265.0
997,#error,NaN,10454.0,NaN
998,#error,NaN,1000.0,NaN


In [6]:
log.drop_duplicates(subset = ['user_id', 'time'])

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
991,Запись пользователя № - user_965,[2019-04-20 12:55:41,800.0,6927.0
992,Запись пользователя № - user_967,[2019-04-20 14:59:36,10154.0,NaN
993,Запись пользователя № - user_973,[2019-04-20 17:09:56,10254.0,NaN
994,Запись пользователя № - user_977,[2019-04-20 18:10:07,10354.0,NaN


### Преобразование к datetime

Вспомним прошлый модуль по очистке данных: признак time был слегка испорчен, в начале каждой даты стоял лишний символ [, вы его убрали.  

Для преобразования столбца с датой в виде текста в дату формата datetime (см. Модуль "С1. Работа с датами"), необходимо использовать метод to_datetime() в библиотеке pandas.

Внутрь метода нужно передать тот столбец, который требуется преобразовать.

Компьютеры понимают числа, но не понимают текст (хотя представители профессии Data Scientist найдут, что сказать по этому поводу). Поэтому, если у вас время записано в виде строки, нужно вытащить из него числа. Таким образом можно получить очень много полезной информации в виде чисел:

Часы, минуты, секунды.
Год, месяц, день.
Более сложные объекты: время дня, время года и другое.
Если мы храним время в виде специальных объектов типа Timestamp, с ним можно проводить и другие операции, например, сложение и вычитание.
Поэтому сейчас важно выполнить преобразование и продолжить работу.

In [7]:
log['time']=log['time'].str.replace('[', '')
log

,user_id,time,bet,win
0,Запись пользователя № - user_919,2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
995,Запись пользователя № - user_984,2019-04-20 9:59:58,9754.0,NaN
996,#error,NaN,10054.0,29265.0
997,#error,NaN,10454.0,NaN
998,#error,NaN,1000.0,NaN


In [22]:
from datetime import datetime, timedelta

log.dropna(axis = 0, how = 'any', subset = ['time'])

df = pd.to_datetime(log['time'], format = '%Y-%m-%d %H:%M:%S')

df.max()

Timestamp('2019-04-20 18:10:07')

In [2]:
log = pd.read_csv("log.xls")  
log = log.dropna()  
log.columns = ['user_id', 'time', 'bet', 'win']  
log['time'] = log['time'].apply(lambda x: x[1:])  
log['time'] = pd.to_datetime(log['time'])  
log['time'] = log.time.apply(lambda x: x.minute)
#log.time = log.time.apply(lambda x: x.minute)
log['time'].head() 

13     57
28     59
150    54
188    34
204    26
Name: time, dtype: int64

### Извлечение признаков времени

В данном блоке мы будем использовать уже знакомые вам возможности pandas: работу со временем и преобразование столбцов.

Для начала вспомним, что мы можем делать с datetime. Вот примеры атрибутов, по которым мы можем обращаться к данным объектам:

year: возвращает год
month: возвращает месяц
day: возвращает день
hour, minute, second - час, минута, секунда
dayofweek - день недели, от 0 до 6, где 0 - понедельник, 6 - воскресенье

Ранее в курсе вы разбирали метод apply(). Он позволяет применить определенную функцию к каждому элементу в столбце.

В метод apply() можно передавать обычные и lambda-функции.

Например, если мы хотим получить столбец, в котором каждым значением будет год из другого столбца (это и есть feature engineering - создание новых признаков из старых), мы можем сделать следующее:

In [ ]:
year_column = log['time'].apply(lambda x: x.year)  

Библиотека pandas позволяет использовать аксессор dt для упрощения подобной работы:

In [ ]:
year_column = log['time'].dt.year

Аксессор - это атрибут столбца, который хранит переменные типа Timestamp, то есть переменные, которые были строковым представлением времени, а затем изменены с помощью pd.to_datetime(). Если вы попытаетесь обратиться к dt у столбца, в котором лежит что-то отличное от времени, вы получите ошибку.

Одним из часто используемых методов в pandas является value_counts().

Этот метод возвращает Series, который содержит количества уникальных элементов.

Например, если мы выполним следующий код:

In [ ]:
test = pd.Series([1, 1, 1, 2, 3, 4, 4])  
test.value_counts() 

Это значит, что число 1 встретилось 3 раза, число 4 встретилось 2 раза, а числа 2 и 3 встретились по одному разу.

value_counts() возвращает значения отсортированными по убыванию.

Если в value_counts() передать значение параметра ascending=True, метод вернет значения, отсортированные по возрастанию.

In [1]:
import pandas as pd

In [27]:
log = pd.read_csv("log.xls")  
log = log.dropna()  
log.columns = ['user_id', 'time', 'bet', 'win']  
log['time'] = log['time'].apply(lambda x: x[1:])  
log['time'] = pd.to_datetime(log['time'])  
#log['time'] = log.time.apply(lambda x: x.minute)
log['time'] = log.time.apply(lambda x: x.month)
#log['time'] = log.time.apply(lambda x: x.year)
#log
log['time'].value_counts()
#log['time'].head()
#log.time.value_counts()


3    57
4    51
2    16
1     9
Name: time, dtype: int64

### Это было слишком просто
Просто, не так ли?

В реальности мы используем эти же методы для получения более интересных вещей из данных.

Новые интересные знания, которые получаются из данных, мы называем инсайтами (insights).

Следующие пара заданий позволят нам получить результаты, которые можно называть инсайтами и которые могут принести реальную пользу.

In [115]:
log = pd.read_csv("log.xls")
log.columns = ['user_id', 'time', 'bet', 'win'] 
log['time']=log['time'].str.replace('[', '')
log['time'] = pd.to_datetime(log['time'])  
log['time'] = log.time.apply(lambda x: x.dayofweek)
log.time.apply(lambda x: x==5 or x==6).sum()



283

In [109]:
log = pd.read_csv("log.xls")
log.columns = ['user_id', 'time', 'bet', 'win'] 
log['time']=log['time'].str.replace('[', '')
log['time']=pd.to_datetime(log['time']) #преобразуем к datatime
year_column = log['time'].apply(lambda x: x.month) 
year_column.value_counts(ascending=True)
# получилось правильный ответ 4 тк создан новый столбец!!!

4.0    170
2.0    259
3.0    264
1.0    291
Name: time, dtype: int64

In [111]:
log = log.loc[~log.time.isna()]
log

,user_id,time,bet,win
0,Запись пользователя № - user_973,2019-01-01 14:51:16,NaN,NaN
1,Запись пользователя № - user_903,2019-01-01 16:31:16,NaN,NaN
2,Запись пользователя № - user_954,2019-01-01 17:17:51,NaN,NaN
3,Запись пользователя № - user_954,2019-01-01 21:31:18,NaN,NaN
4,Запись пользователя № - user_917,2019-01-01 23:34:55,156789.0,NaN
...,...,...,...,...
990,Запись пользователя № - user_965,2019-04-20 12:55:41,800.0,6927.0
991,Запись пользователя № - user_967,2019-04-20 14:59:36,10154.0,NaN
992,Запись пользователя № - user_973,2019-04-20 17:09:56,10254.0,NaN
993,Запись пользователя № - user_977,2019-04-20 18:10:07,10354.0,NaN


In [ ]:
log =log [log ['time'].isna()==False]
#является аналогом кода удаления строк если в строке есть пустые ячейки в конкретном столбце
log.dropna(subset=['time'])
#Вышеописанный код вместо удаления строк просто сначала фильтрует строки без пропусков в конктретном столбце,
#а потом выводит исходный датафрейм на этом фильтре.

In [116]:
log = pd.read_csv("log.xls", header = None)
log.columns = ['user_id','time','bet','win']
# Убираем пропуски
log =log [log ['time'].isna()==False]
# Убираем лишнюю скобку в столбце времени
log['time'] = log.time.apply(lambda t: t[1:])
#Преобразуем time в формат даты и времени
log['time'] = pd.to_datetime(log['time'], format='%Y-%m-%d %H:%M:%S')
log['time'] = pd.to_datetime(log['time'], format='%Y-%m-%A %H:%M:%S')
log['day'] = log['time'].dt.weekday
log['day'].value_counts()
log.time.dt.weekday.apply(lambda x: x==5 or x==6).sum()

283

In [117]:
log = pd.read_csv(PATH_to_file+'log.csv', header=None)
log.columns = ['user_id', 'time', 'bet', 'win']  
log.time.dropna(inplace=True)
log['time'] = log['time'].apply(lambda x: x[1:])  
log['time'] = pd.to_datetime(log['time']) 
log.time.dt.weekday.apply(lambda x: 1 if x in [5,6] else 0).sum()

NameError: name 'PATH_to_file' is not defined

In [119]:
log[log['time'].dt.dayofweek // 5 == 1].time.dt.dayofweek.count()

283